In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import os
import re
import scirpy as ir
import anndata as ad
import decoupler as dc
import scipy as sp
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib_venn import venn2, venn2_circles
import scipy.stats as stats

In [ ]:
### treatment venns

adata_sc_cd4s = adata_sc[adata_sc.obs["fine_clustering"].isin(CD4s)]


saline = adata_sc_cd4s[adata_sc_cd4s.obs["treatment"].isin(["Saline"])]
one_five = adata_sc_cd4s[adata_sc_cd4s.obs["treatment"].isin(["1.5MIU"])]
two_five = adata_sc_cd4s[adata_sc_cd4s.obs["treatment"].isin(["2.5MIU"])]

pre_saline = saline[saline.obs["timepoint"].isin(["pre"])]
pre_one_five = one_five[one_five.obs["timepoint"].isin(["pre"])]
pre_two_five  = two_five[two_five.obs["timepoint"].isin(["pre"])]

post_saline = saline[saline.obs["timepoint"].isin(["post"])]
post_one_five = one_five[one_five.obs["timepoint"].isin(["post"])]
post_two_five  = two_five[two_five.obs["timepoint"].isin(["post"])]

pre_saline = pre_saline.obs["cc_aa_alignment"].unique()
pre_one_five = pre_one_five.obs["cc_aa_alignment"].unique()
pre_two_five = pre_two_five.obs["cc_aa_alignment"].unique()

post_saline = post_saline.obs["cc_aa_alignment"].unique()
post_one_five = post_one_five.obs["cc_aa_alignment"].unique()
post_two_five = post_two_five.obs["cc_aa_alignment"].unique()

shared_saline_clonos = list(set(pre_saline).intersection(post_saline))
shared_one_five_clonos = list(set(pre_one_five).intersection(post_one_five))
shared_two_five_clonos = list(set(pre_two_five).intersection(post_two_five))


figure, axes = plt.subplots(3, 1)
plt.suptitle("Shared Clonotypes (# of clonotypes)")

venn2(subsets=(len(pre_saline), len(post_saline), len(shared_saline_clonos)), 
      set_labels = ("Sal Pre", "Sal Post"), ax=axes[0])
venn2(subsets=(len(pre_one_five), len(post_one_five), len(shared_one_five_clonos)), 
      set_labels = ("1.5 Pre", "1.5 Post"), ax=axes[1])
venn2(subsets=(len(pre_two_five), len(post_two_five), len(shared_two_five_clonos)), 
      set_labels = ("2.5 Pre", "2.5 Post"), ax=axes[2])

font = {"family": "Times New Roman", "size": 12}
plt.rc("font", **font)
plt.show()

figure, axes = plt.subplots(3, 1)
plt.suptitle("Shared Clonotypes (# of cells)")

venn2(subsets=(len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(pre_saline)]), 
               len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(post_saline)]),
                             len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(shared_saline_clonos)])), 
      set_labels = ("Sal Pre", "Sal Post"), ax=axes[0])

venn2(subsets=(len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(pre_one_five)]),  
               len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(post_one_five)]),
                             len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(shared_one_five_clonos)])),
      set_labels = ("1.5 Pre", "1.5 Post"), ax=axes[1])

venn2(subsets=(len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(pre_two_five)]), 
               len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(post_two_five)]),
               len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(shared_two_five_clonos)])),
      set_labels = ("2.5 Pre", "2.5 Post"), ax=axes[2])

font = {"family": "Arial", "size": 12}
plt.rc("font", **font)
plt.show()

figure, axes = plt.subplots(3, 1)
plt.suptitle("Shared Clonotypes (# of cells, percentage)")

total_sal = int(len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(pre_saline)]) + len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(post_saline)]))
total_1_5 = int(len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(pre_one_five)]) + len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(post_one_five)]))
total_2_5 = int(len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(pre_two_five)]) + len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(post_two_five)]))

pre_sal_p = round((len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(pre_saline)])/total_sal)*100, 2)
post_sal_p = round((len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(post_saline)])/total_sal)*100, 2)
shared_sal_p = round((len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(shared_saline_clonos)])/total_sal)*100, 2)

pre_one_five_p = round((len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(pre_one_five)])/total_1_5)*100, 2)
post_one_five_p = round((len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(post_one_five)])/total_1_5)*100, 2)
shared_one_five_p = round((len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(shared_one_five_clonos)])/total_1_5)*100, 2)

pre_two_five_p = round((len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(pre_two_five)])/total_2_5)*100, 2)
post_two_five_p = round((len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(post_two_five)])/total_2_5)*100, 2)
shared_two_five_p = round((len(adata_sc_cd4s[adata_sc_cd4s.obs["cc_aa_alignment"].isin(shared_two_five_clonos)])/total_2_5)*100, 2)

venn2(subsets= (pre_sal_p, 
               post_sal_p,
               shared_sal_p), 
      set_labels = ("Sal Pre", "Sal Post"), ax=axes[0])

venn2(subsets=(pre_one_five_p,  
               post_one_five_p,
               shared_one_five_p),
      set_labels = ("1.5 Pre", "1.5 Post"), ax=axes[1])

venn2(subsets=(pre_two_five_p,
               post_two_five_p,
               shared_two_five_p),
      set_labels = ("2.5 Pre", "2.5 Post"), ax=axes[2])

font = {"family": "Times New Roman", "size": 12}
plt.rc("font", **font)
plt.show()